In [1]:
# if you want to have a pop up sign in method instead please uncomment  on ee.Authenticate and ee.Initialize, but comment on below

#import ee

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
#ee.Initialize()

# here we try to use json api information instead (service account) - comment below if you use above methode
import ee
service_account = 'iqbalpythonapi@bukit30project.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, './bukit30project-4d92e5b46ea7.json')
ee.Initialize(credentials)

In [2]:
import geemap#.foliumap as geemap
import geopandas as gpd
import numpy as np
from sklearn.metrics import confusion_matrix

#Algorith outcome input (Forest Basemap (High baseline))
algor_process = 'Hansen'  #look below
# Enter-input 'Sentinel' for prioritizing 'Sentinel' to get have Forest (High Baseline) and fill in the 10 years rule by Hansen,
# Enter-input 'Hansen' for prioritizing TCL based forest (High Baseline), and 10 years rule and fill in by Sentinel for the uncertain areas (No Changes in areas that Crown cover)

eval_process = False

calc_areas = True

##### INPUT ###########################
title_map = 'AOI_B30'

#AOI File (SHP) - added shp_local: please make a folder in the main directory: shp_local and put here as an input (example) - shp local not publish to github
AOIt_shp = './shp_local/AOI_B30.shp'
AOIt_shp_plot = geemap.shp_to_ee(AOIt_shp)
crs_input = 'EPSG:4326'

################
#for area id in shapefile that identified the data, and will converted into raster
OID = 'id'  #IMPORTANT TO CHECK OID based on the column ID
#############################################

AOI = AOIt_shp_plot

# Accuracy point Reference
ref_shp = './shp_local/ref_point_B30.shp' #### CHANGE THIS AS PER AOI LOCATION
#reference point input to gee object and geopandas
ref_point_loc = geemap.shp_to_ee(ref_shp)
gdf_ref = gpd.read_file(ref_shp)
#print(gdf_ref.head())

######## HANSEN ##################
# Hansen 10 years rule and Forest - Tree Cover Hansen
#define tree cover minimum that classified as forest e.g., Indonesia > 30%, hence 30
tree_cover_forest = 30 
pixel_number = 3 #define minimum mapping unit classified as forest, 1 pixel for landsat 30mx30, 3 pixel is forest ~> 0.27 Ha
year_start_loss = 12 #define start year to track as 10 years rule (e.g., 2012 to 2022 (track 10 years rule), hence 12. format= 00->2000, 12->2012)

#CloudLess Images Sentinel
START_DATE = '2022-1-1'   #Change this as perusal
END_DATE = '2022-12-31'

#Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)
#Map.addLayer(AOI_img) #check if it works
################

#FOR FOREST DETECTION - Sentinel

# Colors fo raster symbology
water = '#3380cc' #Blue 
hi_forest = '#006837' #Strong green
low_forest = '#3ea540' #Medium green
grass_land = '#baf096' #Light green
bare_land = '#ad8855' #Brown
other = '#000000'

#COEFICIENTS: These coeficients are orientative
#and some tweak may be needed depending on the
#location and case of study

#NDWI water limit
ndwi_hi = 0.05

#Bare soil index (BI), soil limit
bi_hi = 2

#NDVI high and low limits
ndvi_lo = 0.25     #0.20 for L1C (suggested value)
                   #0.25 for L2A (suggested value)
ndvi_hi = 0.45     #0.40 for L1C (suggested value)
                   #0.45 for L2A (suggested value)

#Shadow index (SI) high and low limits
si_lo = 0.92       #0.90 for L1C (suggested value) 
                   #0.92 for L2A (suggested value)
si_hi = 0.95       #0.93 for L1C (suggested value) 
                   #0.95 for L2A (suggested value)
#############################

# output Band Names
band_name_image = 'Class'

##################################################################################

In [3]:
############################ Deforestation areas - TCL
# hansen - updated version - global data
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

#Canopy cover percentage (e.g. 30%), for Indonesia
cc = ee.Number(tree_cover_forest)

#Minimum forest area in pixels (e.g. 3 pixels, ~ 0.27 ha in this example).
pixels = ee.Number(pixel_number)

#Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = pixels

canopyCover = gfc.select(['treecover2000'])
canopyCoverThreshold = canopyCover.gte(cc).selfMask()

#Use connectedPixelCount() to get contiguous area.
contArea = canopyCoverThreshold.connectedPixelCount()
#Apply the minimum area requirement.
minArea = contArea.gte(pixels).selfMask()

prj = gfc.projection()
scale = prj.nominalScale()
ForestArea2000Hansen = minArea.reproject(prj.atScale(scale))

Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)
Map.addLayer(ForestArea2000Hansen, {
    'palette': ['#96ED89']
}, 'tree cover: >= min canopy cover & area (light green)',False)

#create visual boundary color only
empty = ee.Image().byte()
AOIm = empty.paint(AOI,0,10)

#Map.addLayer(AOIm,{'palette': '#e043f3'},'AOI') #still buggy
#Map.addLayer(AOIm,{},'AOI')

'''
forestArea = minArea.multiply(ee.Image.pixelArea()).divide(10000)
forestSize = forestArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)

print(
    'Year 2000 tree cover (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',
    forestSize.get('treecover2000'))
    
'''

'''
pixelCount = minArea.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)
onePixel = forestSize.getNumber('treecover2000').divide(pixelCount.getNumber('treecover2000'))
minAreaUsed = onePixel.multiply(pixels)
print('Minimum forest area used (ha)\n ', minAreaUsed)
'''

"\npixelCount = minArea.reduceRegion(\n    reducer=ee.Reducer.count(),\n    geometry=AOI.geometry(),\n    scale=30,\n    maxPixels=1e13\n)\nonePixel = forestSize.getNumber('treecover2000').divide(pixelCount.getNumber('treecover2000'))\nminAreaUsed = onePixel.multiply(pixels)\nprint('Minimum forest area used (ha)\n ', minAreaUsed)\n"

In [4]:
##### Visualized LANDSAT CLOUDLESS FROM HANSEN DATASET LAST YEAR
LastImageLandsat = gfc.select(['last_b30', 'last_b40', 'last_b50', 'last_b70']).rename(['red', 'nir', 'swir1', 'swir2']).clip(AOI)
Map.addLayer(LastImageLandsat, {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 600, 'gamma': 1.5 },
                f'Landsat Hansen Last Year {END_DATE}')

In [5]:
### HISTORICAL DATA THRESHOLD - Get the TCL data LOSS PIXEL overall data from loss year in the input

treeLossYear = gfc.select(['lossyear'])
treeLoss = treeLossYear.gte(year_start_loss).selfMask() # tree loss in e.g., year > 2012 ####SHOULD CHANGE TO RECENT YEAR for the '12' number
#Select the tree loss within the derived tree cover
#(>= canopy cover and area requirements). THIS ONE ALREADY MASK TCL IN FOREST AREA
treecoverLoss = minArea.And(treeLoss).rename(f'lossfrom_{year_start_loss}').selfMask()

#Create connectedPixelCount() to get contiguous area.
contLoss = treecoverLoss.connectedPixelCount()
#Apply the minimum area requirement, and get the TCL data ---> minLoss - ACTUAL TCL AREA from Hansen since the year_start_loss
minLoss = contLoss.gte(lossPixels).selfMask()

'''
lossArea = minLoss.multiply(ee.Image.pixelArea()).divide(10000) #hectare
lossSize = lossArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)
print(
    '>Year 2012 tree loss (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',
    lossSize.get(f'lossfrom_{year_start_loss}'))
'''


"\nlossArea = minLoss.multiply(ee.Image.pixelArea()).divide(10000) #hectare\nlossSize = lossArea.reduceRegion(\n    reducer=ee.Reducer.sum(),\n    geometry=AOI.geometry(),\n    scale=30,\n    maxPixels=1e13\n)\nprint(\n    '>Year 2012 tree loss (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',\n    lossSize.get(f'lossfrom_{year_start_loss}'))\n"

In [6]:
##### START SENTINEL CLOUDLESS COMPOSITE AND MOSAICKING

#CLOUD_FILTER = 80
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

#s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 30})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

#s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

# Mosaic the image collection
#img_mosaic = s2_sr_cld_col_eval_disp.mosaic()

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

SEN_BANDS = ['B2',   'B3', 'B4',  'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
bandNamesSentinel2 = ['blue', 'green', 'red', 'redE1', 'redE2', 'redE3', 'nir', 'redE4', 'swir1', 'swir2']

s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .select(SEN_BANDS, bandNamesSentinel2)
                             .median())


In [7]:
# VISUALIZE SENTINEL CLOUDLESS
Map.addLayer(s2_sr_median.clip(AOI),
                {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 6000, 'gamma': 1.1},
                'S2 cloud-free mosaic')

Map.addLayer(treeLossYear.clip(AOI),{"opacity":1,"bands":["lossyear"],"min":1,"max":21,"palette":["3d358c","4457c9","4777f0","4196ff","2eb4f3","1ad1d5","1ae5b6","36f493","64fd6a","92ff47","b4f836","d3e935","ecd239","fbb938","fe992c","f9751d","ec520e","d93806","bf2102","9f1001","7a0403"]},"Tree Loss Year",False)
Map.addLayer(minLoss.clip(AOI),{
    'palette': ['#ff0000']
}, f'tree cover >30% loss since 20{year_start_loss}',False)

In [8]:
# Function to use Sentinel data for generating Forest, from NDVI, BI, SI
def applying_sentinel_forest(s2_sr_median,ndvi_hi,bi_hi,si_hi,pixels): 
    ############# SPECTRAL INDICES #############################
    #NDVI - NORMALIZED DIFFERENCES VEGETATION INDEX
    ndvi = s2_sr_median.normalizedDifference(['nir','red'])
    #BARELAND/ BARE SOIL INDEX
    bi_1 = s2_sr_median.expression(
        '(NIR + GREEN + RED) / (NIR + GREEN - RED)', 
            {
          'NIR': s2_sr_median.select('nir').divide(10000),
          'GREEN': s2_sr_median.select('green').divide(10000),
          'RED': s2_sr_median.select('red').divide(10000)
            })
    #SOIL INDEX
    si = s2_sr_median.expression(
         '((1 - GREEN) * (1 - RED) )**(0.5)' ,
        {
          'GREEN': s2_sr_median.select('green').divide(10000),
          'RED': s2_sr_median.select('red').divide(10000)
        })

    #Map.addLayer(ndvi,{},'NDVI')
    #Map.addLayer(bi_1,{},'BI_1')
    #Map.addLayer(si,{},'SI')
    ############################################################

    ########### DECISION ON THE DEFINITION
    # NDVI > NDVI_High threshold
    ndviHi_masked = ndvi.updateMask(ndvi.gt(ndvi_hi))
    # BI < BI_High threshold
    bi_1Lo_masked = bi_1.updateMask(bi_1.lt(bi_hi))
    # Shadow Index (SI)> SI threshold
    siHi_masked = si.updateMask(si.gt(si_hi))

    #### HIGH DENSITY FOREST DECISION
    # If ndvi is high enough, baresoil index is low and shadow index is high, then it is a high density forest based on this
    hiforest_masked = ndviHi_masked.And(bi_1Lo_masked).And(siHi_masked)
    #Create connectedPixelCount() to get contiguous area, minimum mapping unit
    conthiForest = hiforest_masked.connectedPixelCount()
    #Apply the minimum area requirement, for the final result one area should have at least 
    hiforest_masked = conthiForest.gte(pixels).selfMask()
    
    #Map.addLayer(hiforest_masked.clip(AOI),{'palette':hi_forest},'High Forest',False)

    '''
    #Low density Forest - not used anywhere
    ndviMid_masked = ndvi.updateMask(ndvi.lt(ndvi_hi).And(ndvi.gt(ndvi_lo)))
    siMid_masked = si.updateMask(si.lt(si_hi).And(si.gt(si_lo)))

    loforest_masked = ndviMid_masked.And(bi_1Lo_masked).And(siMid_masked)
    #Map.addLayer(loforest_masked,{'palette':low_forest},'Low Forest')
    '''
    return hiforest_masked.clip(AOI)


In [9]:
# Utils
def assigning_band(band_name_image,class_value,srcImg):
        
    # Create an image with the constant value for class
    constant_image_class = srcImg.multiply(0).add(class_value).rename(band_name_image)
    constant_image_pixel = srcImg.multiply(0).add(1).rename('pixel')
    
    # Add the new band to the existing image
    pixel_bandimg= srcImg.addBands(constant_image_pixel)
    pix_classImg = pixel_bandimg.addBands(constant_image_class)
    pix_classImg = pix_classImg.select([band_name_image,'pixel'])
    
    return pix_classImg

In [10]:
## VERSION 1 for Sentinel as forest basemap, and followed by Hansen data - in function ####################################
hiforest_masked = applying_sentinel_forest(s2_sr_median,ndvi_hi,bi_hi,si_hi,pixels)

def applying_sentinel_water(s2_sr_median, ndwi_hi):

    # Normalized Differences Water Index
    ndwi = s2_sr_median.normalizedDifference(['green','nir'])
    #Map.addLayer(ndwi,{},'NDWI')

    # Mask the non-watery parts of the image, where NDWI < ndwi_hi.
    waterMasked = ndwi.updateMask(ndwi.gt(ndwi_hi))
    #Map.addLayer(waterMasked, {'palette': water}, 'NDWI (Water) masked')
    
    ##### Get the area only that is not possible for planting (Water)
    waterinAOI = waterMasked.And(AOI_img)
    #Map.addLayer(waterinAOI,{'palette': water},"Water in AOI")
    water_edited = ee.Image(assigning_band(band_name_image,5,waterinAOI))
    return [waterinAOI,water_edited]

def applying_zone_sentinel_hansen(hiforest_masked,minLoss,s2_sr_median, ndwi_hi):
    
    ######### Zone 2 - High Baseline Forest: That does not have (overlaid with) TCL 
    # Unmasked the forest loss 10 years rule
    unmaskedLoss = AOI_img.unmask().updateMask(minLoss.mask().Not()).clip(AOI)
    highBaselineF = hiforest_masked.And(unmaskedLoss)
    #Map.addLayer(highBaselineF,{'palette':hi_forest},'High baseline (Forest)')
    
    # assigning class band 2
    highf_edited = ee.Image(assigning_band(band_name_image,2,highBaselineF)) #high baseline forest - 2
    ##################################################
    
    ######### Zone 3: High Forest and Loss (e.g., Young Regenerating Forest)
    ####### Get the overlay information of HighBaseline (Sentinel) and Tree Cover Loss (Hansen), e.g., Young Regenerating Forest
    HiForestAndLoss = AOI_img.And(hiforest_masked.And(minLoss)) #minLoss is the actual TCL without overlaying with Sentinel
    #Map.addLayer(HiForestAndLoss,{'palette': '#FF0000'},"High Baseline and not pass 10 years rule")
    
    # Final Result for High Forest and TCL -> Class number: 3
    tenyrfl_edited = ee.Image(assigning_band(band_name_image,3,HiForestAndLoss)) #tenyears rule not pass and high baseline - 3
    ##############################################
    
    ######### Zone 4: Tree Cover Loss on the current low baseline (Sentinel)
    ###### Get the 10 years data only that is not overlay with Sentinel High baseline (Forest) #############
    # Create a helper mask indicating where the smaller areas maskhiFL, distinguish only the highbaseline and TCL (mask) and assign mask as 1
    maskHiFL = HiForestAndLoss.mask()
    #Map.addLayer(maskHiFL)
    # Helper - Invert the mask to get the areas where the smaller raster is absent - get only area outside high forest and loss, and mask from 1 to 0
    # invert mask just to get the number from 0 to 1 or vice versa -> .Not()
    maskHiFL_inverted = maskHiFL.Not()
    #Map.addLayer(maskHiFL_inverted)
    # Unmask the bigger raster in the areas to get the pixel value for area 'outside' HiFL (High Baseline and Forest Loss)
    unmaskedHiFL = AOI_img.unmask().updateMask(maskHiFL_inverted).clip(AOI)
    #Map.addLayer(unmaskedHiFL)
    #outcome result for 10 years data only that is not overlay with Sentinel High baseline (Forest) unmaskedHiFL is Area that is not Forest Sentinel############
    tenYearsRule = unmaskedHiFL.And(minLoss)
    #Map.addLayer(tenYearsRule,{'palette': '#FFA500'},"10 Years Rule - not OK")
    
    # assign the Class into no. 4 - Final Result
    tenyrule_edited = ee.Image(assigning_band(band_name_image,4,tenYearsRule)) #tenyears rule not pass - 4
    ###############################################
    
    ######### Zone 5: Water Body - NDWI Sentinel
    # from a function, output is already assigned in a band name: Class : 5
    water_list = applying_sentinel_water(s2_sr_median, ndwi_hi)
    waterinAOI = water_list[0]
    water_edited = water_list[1]
    
    ###############################################


    ######### Zone 1: Go Zone - outside (forest, forest - TCL, no-forest TCL, WaterBody)
    # Utils - Similar with above (but easier), unmasked the high forest means that in the end only include area that is not in forest (hiforest_masked)
    maskHiF = hiforest_masked.mask()
    maskHiF_inverted = maskHiF.Not()
    # Unmasked High Forest - the result is all the area outside of hiforest_masked (Total all forest), and now in no forest
    unmaskedHiF = AOI_img.unmask().updateMask(maskHiF_inverted).clip(AOI)
    # unmasked the water
    unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)
    
    goZone = unmaskedLoss.And(unmaskedHiF).And(unmaskedWaterAOI)
    #Map.addLayer(goZone,{'palette':'#FFFF00'},'Go Zone')
    # assigning band - Go Zone
    goZone_edited = ee.Image(assigning_band(band_name_image,1,goZone)) #go Zone - 1
    ##############################################
    
    Map.addLayer(goZone_edited,{'bands': ['Class'],'palette':'#FFFF00'},'Go Zone')
    Map.addLayer(highf_edited,{'bands': ['Class'],'palette':hi_forest},'High baseline (Forest)')
    Map.addLayer(tenyrfl_edited,{'bands': ['Class'],'palette': '#FF0000'},"High Baseline and not pass 10 years rule")
    Map.addLayer(tenyrule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK")
    Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI")
    
    labels = ['High Baseline (Forest)', '10 Years Rule not OK', 'High Baseline not passed 10 years rule', 'Water', 'Go Zone']
    # colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
    colors = ['#006837', '#FFA500', '#FF0000', '#3380cc', '#FFFF00']
    # legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

    Map.add_legend(title='Legend', labels=labels, colors=colors)

    ''' mosaicking into single image  - still does not work
    #image_list = ee.List([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])
    # Extract the spatial information from one of the individual images.
    projection = goZone_edited.projection()
    scale = goZone_edited.projection().nominalScale()

    # Create an ee.ImageCollection from the list of images
    image_collection = ee.ImageCollection([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])

    # Merge the images into a single ee.Image
    zoning_final = image_collection.mosaic()
    '''
    return [goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]


In [11]:
def eval_accuracy(band_names_ref, band_name_image,high_forest_image ):
    
    ref_values = gdf_ref[band_names_ref].values
    print("Reference_values: ", ref_values)
    # Select the band of interest from the image to know what classified 1 Forest (High baseline), 0 (outside)
    #band_name_image = 'Class'
    band = high_forest_image.select([band_name_image])
    
    # Unmask the band, replacing masked values (null) - Class: null with 0
    unmasked_band = band.unmask(0)
    
    # Use sampleRegions to extract pixel values at the point locations
    sampled_points = unmasked_band.sampleRegions(
        collection=ref_point_loc,
        scale=10
    )

    # Get the pixel values as a list - another way to get the ref_values
    # ref_values = sampled_points.aggregate_array('Forest_Sen').getInfo()

    # Get the pixel values as a list
    pixel_values = sampled_points.aggregate_array(band_name_image).getInfo()

    # Print the pixel values
    print("Pixel Values: ", pixel_values)
    
    confusion_mat = confusion_matrix(ref_values, pixel_values)
    print("Confusion matrix summary: \n", confusion_mat)
    
    tn, fp, fn, tp = confusion_matrix(ref_values, pixel_values).ravel()
    
    print("TN:",tn,"\n FP:",fp, "\n FN:",fn, "\n TP:",tp)
    
    accuracy = ((tp + tn) / (tp + tn + fp + fn))*100.00
    print("Accuracy: (%) ",accuracy)
    
    precision = (tp / (tp + fp))*100.00
    print("Precision: (%) ",precision)
    
    prod_acc = (tp / (tp + fn))*100.00
    print("Producer Accuracy: (%) ",prod_acc) #means that how is the raster result (image) predict forest only,
    
    user_acc = (tn / (tn + fp))*100.00
    print("User Accuracy: (%) ",user_acc)
    return [tn, fp, fn, tp]
    

In [12]:
''' What confusion matrix look like
                 Predicted Class
               | Forest (1) | No Forest (0) |
True Class     |------------|--------------|
Forest (1)     |    TP      |      FN      |
No Forest (0)  |    FP      |      TN      |

Here is the definition of accuracy and precision

Accuracy: The overall accuracy of the classification.

Accuracy = (TP + TN) / (TP + TN + FP + FN)

Precision: The proportion of correctly classified positive samples (class 1) out of all samples classified as positive.

Precision = TP / (TP + FP)

Producer Accuracy (also known as Recall or Sensitivity): The proportion of correctly classified positive samples (class 1) out of all actual positive samples.

Producer Accuracy = TP / (TP + FN)

User Accuracy (also known as Specificity): The proportion of correctly classified negative samples (class 0) out of all actual negative samples.

User Accuracy = TN / (TN + FP)

'''

' What confusion matrix look like\n                 Predicted Class\n               | Forest (1) | No Forest (0) |\nTrue Class     |------------|--------------|\nForest (1)     |    TP      |      FN      |\nNo Forest (0)  |    FP      |      TN      |\n\nHere is the definition of accuracy and precision\n\nAccuracy: The overall accuracy of the classification.\n\nAccuracy = (TP + TN) / (TP + TN + FP + FN)\n\nPrecision: The proportion of correctly classified positive samples (class 1) out of all samples classified as positive.\n\nPrecision = TP / (TP + FP)\n\nProducer Accuracy (also known as Recall or Sensitivity): The proportion of correctly classified positive samples (class 1) out of all actual positive samples.\n\nProducer Accuracy = TP / (TP + FN)\n\nUser Accuracy (also known as Specificity): The proportion of correctly classified negative samples (class 0) out of all actual negative samples.\n\nUser Accuracy = TN / (TN + FP)\n\n'

In [13]:
if algor_process == 'Sentinel':
    highForestOnly = ee.Image(assigning_band(band_name_image,1,hiforest_masked.clip(AOI)))
    Map.addLayer(highForestOnly,{'bands': ['Class'],'palette':hi_forest},'High_Forest_SENTINEL_Assigned - Class 1')
    tiff_result = applying_zone_sentinel_hansen(hiforest_masked,minLoss,s2_sr_median, ndwi_hi)
    band_names_ref = 'Forest_Sen' #this band_names refer to the name of column in shapefile - which column that refer as point reference value
    if eval_process: 
        eval = eval_accuracy(band_names_ref, band_name_image,highForestOnly)
    
elif algor_process == 'Hansen':
    band_names_ref = 'Forest_Lan' #this band_names refer to the name of column in shapefile - which column that refer as point reference value
     
    ## Version 2 - Hansen - Sentinel
    
    ## Zone 4: 10 Years rule not OK - > CC overlay with TCL, (Tree Crown Cover) more than the threshold in AOI
    tenYearsRule = minLoss.clip(AOI)
    tenYearsRule_edited = ee.Image(assigning_band(band_name_image,4,tenYearsRule))
    Map.addLayer(tenYearsRule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK")
    
    
    ## Zone 3: Forest (Sentinel) passed 10 years rule not OK - TCL before year_start_loss overlay with > CC
    # Direct process to get the < year_rule TCL (Explicit)
    #treeLossYear = gfc.select(['lossyear'])
    treeLoss_lessYear = treeLossYear.lt(year_start_loss).selfMask()
    CC_lessYearloss_overlay = AOI_img.And(minArea.And(treeLoss_lessYear))
    # obtain area ten years < CC with the forest masked (Sentinel)
    ForestRegenerating_lessYear = hiforest_masked.And(CC_lessYearloss_overlay)
    ForestRegenerating_lessYear_edited = ee.Image(assigning_band(band_name_image,3,ForestRegenerating_lessYear))

    Map.addLayer(ForestRegenerating_lessYear_edited,{'bands': ['Class'],'palette': '#FF0000'},f"Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}")
   
    # Zone 1: Go Zone - No Forest (Sentinel) < start_loss_year in CC
    hiFMasked = hiforest_masked.mask()
    maskedHiFI_inverted = hiFMasked.Not()
    unmaskedHiF = AOI_img.unmask().updateMask(maskedHiFI_inverted).clip(AOI)
    GoZoneNoForest_TCL_lessYear = unmaskedHiF.And(CC_lessYearloss_overlay)
    GoZoneNoForest_TCL_lessYear_edited = ee.Image(assigning_band(band_name_image,1,GoZoneNoForest_TCL_lessYear))

    Map.addLayer(GoZoneNoForest_TCL_lessYear_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) in TCL less year >CC")

    ''' #Implying the same as year < year_start_loss and minArea
    ## Zone 3: Forest (Sentinel) 10 years rule not OK - TCL Overlaid with < CC, (Tree Crown Cover) less than the threshold in AOI
    # helper - mask
    tenYearsRule_mask = tenYearsRule.mask()
    masktenYearsRule_inverted = tenYearsRule_mask.Not()
    # Unmasked ten years all - all area outside ten years rule (> CC)
    unmaskedtenYrRuleALL = AOI_img.unmask().updateMask(masktenYearsRule_inverted).clip(AOI)
    # tree loss in the AOI
    tenYearsRuleAll = AOI_img.And(treeLoss)
    # obtain the area in outside ten years rule (>CC) intersected with all ten years rule -> tcl in < CC
    tenYearsRule_notCC = tenYearsRuleAll.And(unmaskedtenYrRuleALL)
    # obtain area ten years < CC with the forest masked (Sentinel)
    Forest10Years_noCC = hiforest_masked.And(tenYearsRule_notCC)
    Forest10Years_noCC_edited = ee.Image(assigning_band(band_name_image,3,Forest10Years_noCC))

    Map.addLayer(Forest10Years_noCC_edited,{'bands': ['Class'],'palette': '#32a844'},"Forest (Sentinel) 10 Years < CC")


    # Zone 1: Go Zone - No Forest (Sentinel) in TCL < CC
    hiFMasked = hiforest_masked.mask()
    maskedHiFI_inverted = hiFMasked.Not()
    unmaskedHiF = AOI_img.unmask().updateMask(maskedHiFI_inverted).clip(AOI)
    GoZoneNoForest_TCLnoCC = unmaskedHiF.And(tenYearsRule_notCC)
    GoZoneNoForest_TCLnoCC_edited = ee.Image(assigning_band(band_name_image,1,GoZoneNoForest_TCLnoCC))

    Map.addLayer(GoZoneNoForest_TCLnoCC_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) in TCL < CC")
    '''
    
    ## Zone 2: Forest (Hansen > CC year 2000 Baseline) does not overlay with TCL (ALL)
    # helper - unmasked, get area only outside all the ten years rule TCL (ALL)
    # tree loss in the AOI and exclude no loss
    treeLossAll = treeLossYear.gt(0).selfMask()
    Map.addLayer(treeLossAll,{},'treelossall')
    tenYearsRuleAll = AOI_img.And(treeLossAll)
    masked_tenYearsRuleAll = tenYearsRuleAll.mask()
    masked_inverted_tenYearsRuleAll = masked_tenYearsRuleAll.Not()
    unmasked_tenYearsRuleAll = AOI_img.unmask().updateMask(masked_inverted_tenYearsRuleAll).clip(AOI)
    # get the area that is forest in 2000 and has no TCL (No overlay with TCL ALL)
    Forest_RecentHansen = ForestArea2000Hansen.And(unmasked_tenYearsRuleAll)
    # assigning the number Class - 2
    Forest_RecentHansen_edited = ee.Image(assigning_band(band_name_image,2,Forest_RecentHansen))
    Map.addLayer(Forest_RecentHansen_edited,{'bands': ['Class'],'palette': hi_forest},f"Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE}) to be evaluated")

    # Zone 3 - Forest (Sentinel) < CC (ALL TCL)
    # less than threshold
    canopyCover_lessThreshold = canopyCover.lt(cc).selfMask()
    Forest_Regenerating_lessCC = hiforest_masked.And(canopyCover_lessThreshold)
    Forest_Regenerating_lessCC_edited = ee.Image(assigning_band(band_name_image,3,Forest_Regenerating_lessCC))
    Map.addLayer(Forest_Regenerating_lessCC_edited,{'bands': ['Class'],'palette': '#FF0000'},f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating")

    # Zone 1 - Go Zone area (No Forest (Sentinel) and < CC)
    GoZone_NoForestSen_lessThreshold = unmaskedHiF.And(canopyCover_lessThreshold)
    GoZone_NoForestSen_lessThreshold_edited = ee.Image(assigning_band(band_name_image,1,GoZone_NoForestSen_lessThreshold))
    Map.addLayer(GoZone_NoForestSen_lessThreshold_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) < CC")

    ## Zone 5: Water Body Based on Hansen Data
    WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
    WaterAOI = AOI_img.mask().updateMask(WaterMask)
    # assigning band Class in WaterBody
    water_edited = ee.Image(assigning_band(band_name_image,5,WaterAOI))

    Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI")
    

    if eval_process:
        print(f"EVALUATING ONLY IN FOREST AREA CC > {tree_cover_forest}% THAT GENERATED BY HANSEN 2000 THAT HAS NO TCL")
        ### EVALUATING FOREST CLASS HANSEN
        # CHANGE THE NUMBER Class into 1 as Forest for evaluation purpose only
        Forest_RecentHansen_edited_Class1_Forest = ee.Image(assigning_band(band_name_image,1,Forest_RecentHansen))
        print("Evaluating only Hansen Forest")
        eval = eval_accuracy(band_names_ref, band_name_image,Forest_RecentHansen_edited_Class1_Forest)   
        print("---------------------------- \n")

        ### Evaluating the all end Result Forest Classified (Hansen + Sentinel)

        def get_matrix(image, band_name_image):
            band = image.select([band_name_image])

            # Unmask the band, replacing masked values (null) - Class: null with 0
            unmasked_band = band.unmask(0)

            # Use sampleRegions to extract pixel values at the point locations
            sampled_points = unmasked_band.sampleRegions(
                collection=ref_point_loc,
                scale=10
            )

            pixel_values = sampled_points.aggregate_array(band_name_image).getInfo()

            return pixel_values

        ref_values = gdf_ref[band_names_ref].values

        # prepare for all class into 1, for matrix number assign as forest is 1, no forest is 0, assumed they are not overlapped
        print(f"Evaluating All Forest Classified (Hansen CC > {tree_cover_forest}%) + Sentinel (inGAP) vs Reference value")
        # in the end, if one of them classified forest, then the all result will be class: 1
        ForestRegenerating_lessYear_eval = ee.Image(assigning_band(band_name_image,1,ForestRegenerating_lessYear))   
        Forest_Regenerating_lessCC_eval = ee.Image(assigning_band(band_name_image,1,Forest_Regenerating_lessCC))

        #all_Forest_Hansen_Sentinel = [Forest_RecentHansen_edited_Class1_Forest,ForestRegenerating_lessYear_eval,Forest_Regenerating_lessCC_eval]

        HansenForestOnly_pixValue = get_matrix(Forest_RecentHansen_edited_Class1_Forest, band_name_image)
        print(f"Hansen pix no Loss in CC > {tree_cover_forest}%", HansenForestOnly_pixValue)
        ForestRegenerating_lessYear_pixValue = get_matrix(ForestRegenerating_lessYear_eval, band_name_image)
        print(f"Forest (Sentinel) pix in TCL year < 20{year_start_loss}",ForestRegenerating_lessYear_pixValue)
        Forest_Regenerating_lessCC_pixValue = get_matrix(Forest_Regenerating_lessCC_eval, band_name_image)
        print(f"Forest (Sentinel) pix in CC (year 2000) < {tree_cover_forest}% - Regenerating",Forest_Regenerating_lessCC_pixValue)
        print("------------------------------")

        a = np.array(HansenForestOnly_pixValue)
        b = np.array(ForestRegenerating_lessYear_pixValue)
        c = np.array(Forest_Regenerating_lessCC_pixValue)

        AllForestArr = a + b + c
        AllForestArr_list = AllForestArr.tolist()

        print("Reference Value: ", ref_values)
        print("Pixel Value (All Forest): ",AllForestArr_list)

        confusion_mat_all = confusion_matrix(ref_values, AllForestArr_list)
        print("Confusion matrix summary: \n", confusion_mat_all)

        tn, fp, fn, tp = confusion_matrix(ref_values, AllForestArr_list).ravel()

        print("TN:",tn,"\n FP:",fp, "\n FN:",fn, "\n TP:",tp)

        accuracy = ((tp + tn) / (tp + tn + fp + fn))*100.00
        print("Accuracy: (%) ",accuracy)

        precision = (tp / (tp + fp))*100.00
        print("Precision: (%) ",precision)

        prod_acc = (tp / (tp + fn))*100.00
        print("Producer Accuracy: (%) ",prod_acc) #means that how is the raster result (image) predict forest only,

        user_acc = (tn / (tn + fp))*100.00
        print("User Accuracy: (%) ",user_acc)
        
        
    labels = [f'Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE})', 
          f'Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}', 
          f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating",
          "10 Years Rule - not OK",
          'Water', 
          'Go Zone',
         ]
    # colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
    colors = [hi_forest, 
              '#FF0000', 
              '#FF0000', 
              '#FFA500', 
               water,
              '#FFFF00',
             ]
    # legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

    Map.add_legend(title='Legend', labels=labels, colors=colors)
    

In [14]:
Map.addLayerControl()
Map

Map(center=[-1.0232057340644143, 102.432211047781], controls=(WidgetControl(options=['position', 'transparent_…

No such comm: d274370a5d714d93a4a3389b14cc5ac5


In [ ]:
# Define a function to calculate the area of an image within the AOI
def areas_calculation(img):
    # Calculate the total number of pixels in the ROI
    area2 = img.reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry=AOI,
      crs=crs_input,
      scale=30,
      maxPixels=1e13
    )

    print('area calculated \n ------')

    return ee.Number(area2.get('pixel')).getInfo()

In [ ]:
import pandas as pd

if algor_process == 'Sentinel' and calc_areas:
    # Create a list of images to analyze
    #[goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]
    list_images = [tiff_result[0], tiff_result[1], tiff_result[2], tiff_result[3], tiff_result[4]]

    # Select the 'pixel' band and calculate the area of each pixel in hectares
    list_areas = [areas_calculation(image.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)) for image in list_images]

    # Map the areas_calculation function over the list of images to calculate the area of each image
    #list_areas = ee.List(selected_band_list_images).map(areas_calculation)
    
    
    data = {'Class': [1, 2, 3, 4,5], 
            'Name_Class': ['Go Zone', 'High Forest Baseline', 'High Baseline no pass 10 years rule', 
                           'No pass 10 years rule', 'Water (Un-plantable)'],
            'Area_Ha':list_areas}
    
elif algor_process == 'Hansen' and calc_areas:
    # Create a list of images to analyze
    #[goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]
    list_images = [GoZoneNoForest_TCL_lessYear_edited, 
                   GoZone_NoForestSen_lessThreshold_edited, 
                   Forest_RecentHansen_edited, 
                   ForestRegenerating_lessYear_edited,
                   Forest_Regenerating_lessCC_edited, 
                   tenYearsRule_edited,
                   water_edited
                  ]

    # Select the 'pixel' band and calculate the area of each pixel in hectares
    list_areas = [areas_calculation(image.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)) for image in list_images]

    # Map the areas_calculation function over the list of images to calculate the area of each image
    #list_areas = ee.List(selected_band_list_images).map(areas_calculation)
    
    data = {'Class': [1,1, 2, 3,3, 4,5], 
        'Name_Class': ['Go Zone - No Forest (Sentinel) in TCL less year >CC', 
                       'Go Zone - No Forest (Sentinel) < CC', 
                       f"Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE}) to be evaluated", 
                       f"Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}", 
                       f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating",
                       "10 Years Rule - not OK",
                       'Water (Un-plantable)',
                      ],
        'Area_Ha':list_areas}

In [ ]:
pd.DataFrame.from_dict(data)

In [ ]:
a = np.array([0, 1, 0,0])
b = np.array([1, 0, 0,1])
c = np.array([0, 0, 1,1])

result = a + b + c
print(result.tolist())

In [ ]:
##### EXPORTING LANDSAT HANSEN LAST YEAR

AOI_NAME = title_map

geemap.ee_export_image_to_drive(
    LastImageLandsat, description=f'LandsatLast_{END_DATE}_{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)
####################################

In [ ]:
### EXPORTING SENTINEL
geemap.ee_export_image_to_drive(
    s2_sr_median.clip(AOI), description=f'Sentinel_{START_DATE}_{END_DATE}_{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

In [ ]:
print('a')

In [ ]:
#png_file = './my_map.png'
#Map.to_image(filename=png_file, monitor=1)

html_file = f'./{title_map}.html'
Map.to_html(filename=html_file, title=title_map)

In [ ]:
list_areas

In [ ]:
print(list_areas)

In [ ]:
import pandas as pd
data = {'Class': [1, 2, 3, 4,5], 
        'Name_Class': ['Go Zone', 'High Forest Baseline', 'High Baseline no pass 10 years rule', 
                       'No pass 10 years rule', 'Water (Un-plantable)'],
        'Area_Ha':list_areas}
pd.DataFrame.from_dict(data)

In [ ]:
AOI_NAME = 'SRA_Sentinel'

geemap.ee_export_image_to_drive(
    goZone_edited, description=f'GoZone{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    highf_edited, description=f'HighBaselineForest{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    tenyrfl_edited, description=f'HighBaseline10yrsForest{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    tenyrule_edited, description=f'tenyrsForest{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    water_edited, description=f'Water{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

In [ ]:
'''
goZoneArea = goZone.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2 = goZoneArea.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
goZoneAreaHa = ee.Number(
  area2.get('first')).getInfo()
print(goZoneAreaHa)

highBaselineF_Area = highBaselineF.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_hBF = highBaselineF_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
highBaselineF_AreaHa = ee.Number(
  area2_hBF.get('nd')).getInfo()
print(highBaselineF_AreaHa)

waterinAOI_Area = waterinAOI.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_wtr = waterinAOI_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
waterinAOI_AreaHa = ee.Number(
  area2_wtr.get('nd')).getInfo()
print(waterinAOI_AreaHa)

HiForestAndLoss_Area = HiForestAndLoss.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_hifl = HiForestAndLoss_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
HiForestAndLoss_AreaHa = ee.Number(
  area2_hifl.get('first')).getInfo()
print(HiForestAndLoss_AreaHa)

tenYearsRule_Area = tenYearsRule.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_10yrule = tenYearsRule_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
tenYearsRule_AreaHa = ee.Number(
  area2_10yrule.get('first')).getInfo()
print(tenYearsRule_AreaHa)
'''

In [ ]:
'''
zoning_cls = zoning_final.select(['Class'])
zoning_pixel = zoning_final.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)

zoning_Area_cls = zoning_pixel.addBands(zoning_cls).reduceRegion(
  reducer=ee.Reducer.sum().group(groupField=1),
  geometry=AOI,
  scale=30,
  maxPixels=1e13
  )

statsFormatted = ee.List(zoning_Area_cls.get('groups')) \
  .map(lambda el: ee.Dictionary(el)) \
  .map(lambda d: [ee.Number(d.get('group')), d.get('sum')])

statsDictionary = ee.Dictionary(statsFormatted.flatten())
print(statsDictionary)
'''